In [1]:
from PIL import Image
from matplotlib import pyplot as plt
import numpy as np
import face_recognition
import keras
from keras.models import load_model
import cv2
from io import BytesIO
import IPython.display
import time
import serial

Using TensorFlow backend.


In [2]:
# Use 'jpeg' instead of 'png' (~5 times faster)
def array_to_image(a, fmt='jpeg'):
    f = BytesIO()
    #Convert array to binary stream object
    Image.fromarray(a).save(f, fmt)
    
    return IPython.display.Image(data=f.getvalue())

In [3]:
def get_frame(cam):
    ret, frame = cam.read()
    
    # Natural viewing
    frame = cv2.flip(frame, 1)
    frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    return frame

In [4]:
def get_face_locations(image):
    return face_recognition.face_locations(image)

def draw_box_on_face(image, face_location):
    top, right, bottom, left = face_location
#     w, h, _ = image.shape
#     border = (w + h / 2) / 16
    border = 8
    cv2.rectangle(image, (left - border, top - border), (right + border // 2, bottom + border // 2), (0, 255, 0), border)

In [5]:
def update_webcam(display, image):
    frame = array_to_image(image)   
    display.update(frame)

In [6]:
def prep_for_emotion_detection(image, face_location):
    top, right, bottom, left = face_location
    
    face_image = image[top:bottom, left:right]
    face_image = cv2.resize(face_image, (48, 48))
    face_image = cv2.cvtColor(face_image, cv2.COLOR_RGB2GRAY)
    
    return np.reshape(face_image, [1, face_image.shape[0], face_image.shape[1], 1])

In [7]:
def detect_emotion(img):
    emotion_dict = {'Angry': 0, 'Sad': 5, 'Neutral': 4, 'Disgust': 1, 'Surprise': 6, 'Fear': 2, 'Happy': 3}
    label_map = dict((v,k) for k,v in emotion_dict.items()) 
    
    predicted_class = np.argmax(model.predict(img))
    return label_map[predicted_class]

In [8]:
def send_to_arduino(emotion):
    serial_dict = { 'Happy': b'1', 'Sad': b'2', 'Fear': b'3', 'Disgust': b'4', 'Angry': b'5', 'Neutral': b'6', 'Surprise': b'7', }
    ser.write(serial_dict[emotion])

In [9]:
cam = cv2.VideoCapture(0)

d = IPython.display.display("", display_id=1)
d2 = IPython.display.display("", display_id=2)
d3 = IPython.display.display("", display_id=3)

model = load_model("face_and_emotion_detection/emotion_detector_models/model_v6_23.hdf5")
ser = serial.Serial('/dev/cu.usbmodem142101', 9600)

while True:
    try:
        t1 = time.time()
        
        image = get_frame(cam)
        face_locations = get_face_locations(image)
        
        if len(face_locations) < 1:
            update_webcam(d, image)
            continue
            
        face_location = face_locations[0]
        draw_box_on_face(image, face_location)
        update_webcam(d, image)

        detectable_img = prep_for_emotion_detection(image, face_location)
        emotion = detect_emotion(detectable_img)
        d2.update(IPython.display.HTML(emotion))
        send_to_arduino(emotion)
        
        t2 = time.time()
        s = f"""{int(1/(t2-t1))} FPS"""
        d3.update(IPython.display.HTML(s))
    except KeyboardInterrupt:
        print()
        cam.release()
        IPython.display.clear_output()
        print ("Stream stopped")
        break

Stream stopped
